# Exploration of the Heart Disease Data Set

## Introduction

The dataset is taken from the Cleveland, Ohio database. This dataset is a collaboration with physicians from Hungary, Switzerland, Virginia, and Ohio. It describes the age, sex, and other medical information about the patients that will give information on whether the patient might have heart disease or not. The data set has numerous attributes related to a full medical history of the patient. These attributes include various types of information related to heart disease like chest pain, years as a smoker, family history of heart disease and other related diseases, blood sugar levels, exercise levels, and anginas (chest pain due to reduced blood flow, symptom of coronary artery disease). The last attribute gives a diagnosis of heart disease for that patient, with a value of 0 being a <50% diameter of the vessel narrowing, or a value of 1 being a >50% diameter of the vessel narrowing. This attribute uses angiographys as a way to predict and diagnose heart disease. The values of this field range from 0 to 4 with 0 being no presence and 1-4 a range of presence. An angina is a very clear symptom of heart disease and has various types of anginas (as categorized by the data set with values 1, 2, 3, 4). A typical angina (value = 1) usually lasts for a few minutes and over the counter medicine can make it go away. An atypical angina (value = 2) can get worse over time and may indicate that the person is having a heart attack. A nonanginal pain (value = 3) usually results in pain similar to a normal angina however is due to different causes. Lastly, an asymptomatic angina is when there is an angina occurring, however usually little to no pain is experienced. 

The question we will try to answer is whether a patient will be diagnosed with heart disease. We have a number of predictors and we will use the technique of backward selection to narrow down our predictors to the few attributes that allow us to predict this most accurately.

We will be using the Heart Disease Data set. This database describes 76 attributes but all experiments that have been published use only 14 as a reference. The `num` column describes the presence of heart disease in the patient. The `col_vessels` column indicates the number of major blood vessels colored by flouroscopy. The `slope` column indicates the slope of the peak exercise ST segment (1 = upslope, 2 = flat, 3 = downslope). The `st_depression` column indicates the ST depression induced by exercise relative to rest. The `ex_ang` column indicates whether the patient has exercise induced angina (1 = yes, 0 = no). The `max_hr` column describes inidcates a numerical value of the maximum heart rate achieved. The `rest_ecg` indicates the resting electrocardiac results (0 = normal, 1 = ST-T wave abnormality, 2 = probable/definite left ventricular hypertropy). The `fbs` column inidcates whether the patient has a fasting blood sugar of greater than 120 mg/dl (1 = true, 0 = false). The `chol` column describes the serum cholesterol level in mg/dl per patient. The `rest_bps` indicates the resting blood pressure (mm Hg). The `chest_pain` column describes the chest pain in the patient with varying types of anginas (1 = typical angina, 2 = atypical angina, 3 = nonanginal pain, 4 = asymptomatic). The `sex` column indicates the gender of the patient (1 = male, 0 = female). The `defect` column refers to the defect of the heart (3 = normal, 6 = defect, 7 = reversible defect). Lastly, the `age` column indicates the age of the patient. 

## Preliminary exploratory data analysis

In [1]:
library(tidyverse)
library(tidymodels)
library(repr)
library(MASS)
library(GGally)
library(caret)
options(repr.matrix.max.rows = 6)

── Attaching core tidyverse packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.1     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.4     ✔ rsample      1.1.1
✔ dials        1.2

### Read the dataset, clean and wrangle the data into a tidy format.

Download the dataset from the url and store it into a local file.

In [2]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'

download.file(url, destfile = "data/processed.cleveland.data")

Read the file into the dataframe `heart_disease_original`. We will need to manually specify the column types, which are given on the database website.

In [3]:
heart_disease_original <- read_csv("data/processed.cleveland.data",
                                   # add column names
                                   col_names = c("age",
                                                 "sex",
                                                 "chest_pain", 
                                                 "rest_bps", 
                                                 "chol", 
                                                 "fbs", 
                                                 "rest_ecg", 
                                                 "max_hr", 
                                                 "ex_ang", 
                                                 "st_depression", 
                                                 "slope", 
                                                 "col_vessels", 
                                                 "defect", 
                                                 "num"))

head(heart_disease_original, 10)

Rows: 303 Columns: 14
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): col_vessels, defect
dbl (12): age, sex, chest_pain, rest_bps, chol, fbs, rest_ecg, max_hr, ex_an...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,chest_pain,rest_bps,chol,fbs,rest_ecg,max_hr,ex_ang,st_depression,slope,col_vessels,defect,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
57,0,4,120,354,0,0,163,1,0.6,1,0.0,3.0,0
63,1,4,130,254,0,2,147,0,1.4,2,1.0,7.0,2
53,1,4,140,203,1,2,155,1,3.1,3,0.0,7.0,1


Now, we need to wrangle `heart_disease_original` by removing all rows that contain a question mark, filtering essentially NA values and changing the type of `ca` and `defect` to be doubles.

In [4]:
heart_disease <- heart_disease_original |> 

                    # remove all rows that contain a question
                    # mark (esentially NA values) to supress
                    # warnings from R
                    filter(defect != "?", col_vessels != "?") |>

                    # change columns col_vessels and defect to be numeric
                    # to allow for tidymodels to work properly
                    mutate(col_vessels = as.numeric(col_vessels),
                           defect = as.numeric(defect),
                           num = as_factor(num)) |>

                    # remove rows with NA values
                    na.omit()     


head(heart_disease, 10)

ERROR: Error in parse(text = x, srcfile = src): <text>:1:42: unexpected '>'
1: heart_disease <- heart_disease_original |>
                                             ^


Let's check the distribution for our target variable `num`. 

In [12]:
heart_disease |> 
   group_by(num) |> 
   summarize(count = n()) |>
   # we can find the percent by dividing count for each
   # num by nrow(heart_disease) which gives us the amount
   # of rows in heart_disease
   mutate(percent = 100 * (count/nrow(heart_disease)))

ERROR: Error in parse(text = x, srcfile = src): <text>:1:16: unexpected '>'
1: heart_disease |>
                   ^


It looks like there is an uneven proportion of data for `num`. There is an proportionally high amount of rows where `num` is 0 vs where `num` is 1, 2, 3, or 4. Due to the lack of distribution of data, we will likely not get a very accurate k-nn model. We will solve this problem later by upsampling our training data. For now, let's continue as normal. 

### Split the dataset into training set and testing set.

We will split our dataset `heart_diease` into a testing and training set. We will upsample the training set which will hopefully increase the accuracy of our model.

In [ ]:
# setting a seed which will not be changed for the rest of the project
set.seed(2023)

heart_disease_split <- initial_split(heart_disease, prop = 0.75, strata = num)
heart_disease_training <- training(heart_disease_split)
heart_disease_testing <- testing(heart_disease_split)

head(heart_disease_training)

Now, we will use the `upSample` function from `caret` to upsample all the minority classes in `heart_disease_training` to have the same amount of observations as the majority class. 

In [ ]:
# setting a seed which will not be changed for the rest of the project
set.seed(2023)

heart_disease_training_upsampled <- upSample(x = heart_disease_training[,-ncol(heart_disease_training)],
                                             y = heart_disease_training$num) |>
                                    # upSample changes the name of the column of the factor variable to
                                    # Class so we will change it back to num manually
                                    mutate(num = Class) |>
                                    dplyr::select(-Class)

head(heart_disease_training_upsampled, 160)

Now we will compare the distribution of `num` between our original training data and our upsampled training data.

In [ ]:
heart_disease_training |> 
   group_by(num) |> 
   summarize(count = n()) |>
   mutate(percent = 100 * (count/nrow(heart_disease_training)))

heart_disease_training_upsampled |> 
   group_by(num) |> 
   summarize(count = n()) |>
   mutate(percent = 100 * (count/nrow(heart_disease_training_upsampled)))

As we can see, in `heart_disease_training`, the distribution of `num` is quite uneven which is what we expected given the distribution of `num` in the original data set. However, in `heart_disease_training_upsampled`, the distribution of `num` is exactly 20% for each factor. This indicates to us that upsampling worked properly. 

# Selecting Predictors

### GGPairs

Of the 14 predictors given, we can conclude that `chest_pain` and `ex_ang` are likely to be predictors that help determine between the different types of num. Chest pain is likely related to the presence of heart disease, as people who don't have heart disease likely don't have chest pain either. Moreover, if someone has an exercise-induced angina, it seems likely that they will be more succeptible to heart disease. However, there is no clear decision on whether or not to include the remaining 12 predictors.

Therefore, to choose our predictor variables, we will use `ggpairs` to find the correlation between all the variables and our factor variable `num`. We will then decide which variable seems to be significant between determining between different factors of `num`. 

However, since we have a lot of predictor variables, to make it easy to view 14 different graphs, we will break our dataset into 2 smaller datasets and view them seperately to make it more legible.


In [ ]:
heart_disease_training_1 <- heart_disease_training_upsampled |>
    # select the first 7 columns as well as num 
    dplyr::select(1:7, num)

heart_disease_training_2 <- heart_disease_training_upsampled |>
    # select the remaining columns
    dplyr::select(8:14)

head(heart_disease_training_1)
head(heart_disease_training_2)

In [ ]:
big_plot_1 <- ggpairs(heart_disease_training_1, 
                      # make the text size bigger 
                      upper = list(continuous = wrap("cor", size = 10)),
                      # color the output by our factor variable and make
                      # the graphs translucent
                      ggplot2::aes(colour = num, alpha = 0.2))

big_plot_2 <- ggpairs(heart_disease_training_2, 
                      upper = list(continuous = wrap("cor", size = 10)),
                      ggplot2::aes(colour = num, alpha = 0.2))

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 30)
 
big_plot_1 + # make all text bigger
             theme(text = element_text(size = 50), 
                   # adjust the axis text to be a bit smaller 
                   axis.text = element_text(size = 20))

big_plot_2 + theme(text = element_text(size = 50), 
                   axis.text = element_text(size = 20))

We will mainly be looking at the diagonal curves of these graphs. 

For the first graph, it looks like `age` and `chol` show a signficant difference between different types of `num`. Surprisingly, it doesn't seem like `chest_pain` is a good predictor between the different types of `num`. We can see a big blue spike (correlating to `num = 3`) at high values of `chest_pain` but otherwise, all the curves look quite similar. Furthermore, it's tough to determine whether `rest_ecg` is a good predictor since even though there seems to be seperation between the curves, the curves all follow the same trend; they rise and fall at the same locations. All the other predictors seem to be bad at differentiating between the different curves.

For the second graph, it looks like `max_hr`, `slope`, `col_vessels` and `defect` are good predictors whereas it's tough to determine whether `st_depression` is good or not. It also tends to rise and fall at the same location. 

To qualitatively determine which predictors to use, we will use the method of backward selection.

Section 6.8.3 of the data science textbook briefly mentions the method of forward selection in R. However, the method it shows is far too manual. The more manual we make our program, the more prone it is to errors. Therefore, we will be using the `MASS` library to import a few functions which allow us to do the same thing.

 Therefore, instead of forward selection, we will use backward selection to remove one predictor at a time. Then, we will check if the AIC of the new model goes down. We will remove the predictor which lowers the AIC the most until we get to a point where removing predictors increases AIC. 
 
AIC, or Akaike Information Criterion, is a measure used to evaluate the goodness of fit of a statistical model. It takes into account both the complexity of the model and its performance in explaining the data. The main goal of AIC is to find the best balance between model complexity and its ability to explain the observed data. Lower AIC values indicate better model performance, while higher AIC values indicate worse model performance.

During the backward selection process, we remove the predictor that, when removed, results in the lowest AIC value for the reduced model. We continue to remove predictors one by one and compare the AIC values, stopping when any further removal of predictors would result in an increase in AIC. By removing predictors that contribute less to the model's performance and comparing AIC values, we can identify the combination of predictors that provides the best balance between model complexity and predictive accuracy.

In [ ]:
# the MASS library requires all values to be an integer
heart_disease_training_aic <- heart_disease_training |>
    mutate(num = as.numeric(num))

In [ ]:
# setting a seed which will not be changed for the rest of the project
set.seed(2023)

null <- lm(num ~ 1, data = heart_disease_training_aic) # 1 means no predictor 
full <- lm(num ~ ., data = heart_disease_training_aic) # use all predictors
 
# this function goes from full (all predictors) to null (no predictors) 
# and removes the predictor whose removal decreases the AIC the most
stepAIC(full, scope = list(lower = null, upper=full), 
         data = heart_disease_training_aic,
         direction = 'backward')

Looking at the output of `stepAIC`, it looks like it suggests that `sex` and `st_depression` were indeed good predictors for our model. Moreover, it also suggests the inclusion of `chest_pain` and `ex_ang` as a predictor. Looking at the graph, it does seem fair to not include `ex_ang` however it looks like `chest_pain` is a toss up. We will veer on the side of caution and not include it.

Personally, we still belive that `sex` is not a good predictor due to the visual confirmation given by the `ggpairs` function, so we will use all the predictors that were given to use by the `stepAIC` function except for `sex`.

### Using only training data, summarize the data in one table.

Since we don't know the best value of $k$ to use for the model, we will use `vfold` and the `tune()` function to determine the accuracy of the model at different values of $k$. Firstly, we will create the recipe.

In [ ]:
# setting a seed which will not be changed for the rest of the project
set.seed(2023)

# create recipe for k-nn using the previously mentioned predictors
heart_disease_recipe <- recipe(num ~ age + chest_pain + chol + max_hr + st_depression + slope + col_vessels + defect,
                               data = heart_disease_training_upsampled) |>
                        # normalize data
                        step_scale(all_predictors()) |>
                        step_center(all_predictors())

We will start the $k$ value at 1 and set the upper limit to 70. We chose the upper limit of 70 since it is close enough to 120 (the majority classifier) that our model will naturally start losing accuracy. In an ideal world, we would have the upper limit at exactly the majority classifier but doing `vfold` with 100+ neighbours takes a long computation time.

In [ ]:
# setting a seed which will not be changed for the rest of the project
set.seed(2023)

# we will use a step value of 5 to provide high accuracy while not severly effecting
# computation speed
k_vals <- tibble(neighbors = seq(from = 1, to = 70, by = 5))

# create a standard 5 fold model of the training data
heart_disease_vfold <- vfold_cv(heart_disease_training, v = 5, strata = num)

# create the specification of our k-nn model using tune() for the neighbours value
heart_disease_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
                 set_engine("kknn") |>
                 set_mode("classification")

heart_disease_tune_results <- workflow() |>
  # add our recipe and model
  add_recipe(heart_disease_recipe) |>
  add_model(heart_disease_tune) |>
  # tune the grid to involve vfold and tuned neighbour 
  tune_grid(resamples = heart_disease_vfold, grid = k_vals) |>
  collect_metrics() 

# sort accuracies by the accuracy of the neighbour count
(accuracies <- heart_disease_tune_results |>
  filter(.metric == "accuracy") |>
  arrange(desc(mean)))

# get the k value with the highest accuracy
(best_k_value <- accuracies |>
    dplyr::select(neighbors) |>
    slice(1) |>
    pull())

It looks like a neighbour count of around 41-46 gives us the highest accuracy. To further hone in on the exact neighbour count which gives us the maximal accuracy, we will repeat this process once more with a more precise range for our k values.

In [ ]:
# setting a seed which will not be changed for the rest of the project
set.seed(2023)

# since 51 seems to be in the same range as 41 for the accuracy, we will let this tibble
# extend more to the right of the best_k_value than to the left
k_vals_precise <- tibble(neighbors = seq(from = best_k_value - 10, to = best_k_value + 15, by = 1))


# the rest of this code is essentially copy pasted from the last time we ran this
# workflow, but with updated names; notice that we can reuse our old recipe and 
# specification
heart_disease_tune_results_precise <- workflow() |>
  add_recipe(heart_disease_recipe) |>
  add_model(heart_disease_tune) |>
  tune_grid(resamples = heart_disease_vfold, grid = k_vals_precise) |>
  collect_metrics() 

(accuracies_precise <- heart_disease_tune_results_precise |>
  filter(.metric == "accuracy") |>
  arrange(desc(mean)))

(best_k_value_precise <- accuracies_precise |>
    dplyr::select(neighbors) |>
    slice(1) |>
    pull())

We will now plot the accuracy vs the neighbour count to get a better idea of what this data looks like.

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)

(accuracy_vs_k_precise <- ggplot(accuracies_precise, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors", y = "Accuracy Estimate") + 
  theme(text = element_text(size = 12)))

It looks like $k = 43, 44$ gives the highest accuracy value, which tapers off quickly and rising again just as quickly for another peak at around 49. From there it tapers off to a lower accuracy where it seems to stay. We will use the lower value of 43 just for computation's sake.

### Final Model with Known K-Value

In [ ]:
# setting a seed which will not be changed for the rest of the project
set.seed(2023)

knn_spec <- nearest_neighbor(weight_func = "rectangular", 
                             neighbors = best_k_value_precise) |>
  set_engine("kknn") |>
  set_mode("classification")

knn_fit <- workflow() |>
  add_recipe(heart_disease_recipe) |>
  add_model(knn_spec) |>
  fit(data = heart_disease_training)

In [ ]:
# setting a seed which will not be changed for the rest of the project
set.seed(2023)

heart_disease_test_predictions <- predict(knn_fit, heart_disease_testing) |>
  bind_cols(heart_disease_testing)

(heart_disease_test_predictions |>
  metrics(truth = num, estimate = .pred_class) |>
  filter(.metric == "accuracy"))

It looks like our final model accuracy is 59%. If we remember from before, our majority classifier was `num = 0` which made up of 54% of the original data. This means that our model's accuracy improved upon just guessing every patient to have no heart disease by 5%. This may not seem like much, but due to the un-uniformity of our data, it makes sense that we did not achieve an incredibly accurate model.

Let's visualize how our confusion matrix looks like.

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)

confusion <- heart_disease_test_predictions |>
             conf_mat(truth = num, estimate = .pred_class)

autoplot(confusion, type = "heatmap") +
  scale_fill_gradient(low = "red", high = "green") +
  ggtitle("Confusion Matrix") +
  # make all text bigger
  theme(
      axis.title.x = element_text(size = 14),
      axis.title.y = element_text(size = 14),
      plot.title = element_text(size = 20)
  )

It looks like our model is extremely accurate at guessing when the patient has no heart disease. There was only 1 case where the patient did not have heart disease but our model assumed they did. However, where our model struggles is classifying patients with heart disease. For a `num` value of 2 and 3, our model only correctly classified once whereas it was never accurate in correctly classifying a `num` value of 4. This makes sense, since we had an under-representation of the `num` values of 1, 2, 3, and 4 in our original data which we had to compensate for by upsampling.

## Expected outcomes and significance

- What do you expect to find?
- What impact could such findings have?
- What future questions could this lead to?

Our model used the k-nearest neighbours algorithm with the data set of Cleveland heart disease patients to come up with an accuracy of 59% when determining the presence or lack thereof of heart disease (and the type of heart disease detected). Since the majority classifier in the data was 54%, our model improved on the majority classifier by 5%. If we look at the confusion matrix, our model is very good at determining cases where the patient had no heart disease. This was to be expected given the oversampling of cases in the data set where the patient had no heart disease.

One way to fix the accuracy of this model is to first determine whether a patient has heart disease or not. This could be determined by changing all the values of `num = 1, 2, 3, 4` to `num = 1`. Then, we can create a k-nn model which determines the boolean value of whether a patient has heart disease or not. If the model classifies that the patient does indeed have heart disease, then we could run those patients through another model trained on the original data (with all the `num = 0` cases filtered out). Therefore, our second model is only trained on heart disease cases and does not have the extra noise of the no heart disease case and will therefore predict with higher accuracy what severity of heart disease a patient has. 

These findings can be important to future studies or can be put to use at current medical institutions. However, we would need for this model to be a lot more accurate than 59% before we start seeing real life usage of it. We believe that firstly, more data needs to be collected. Somewhere in the level of `n = 1000-5000`. Then, we can possibly see a highly accurate models which could be used to quickly classify patients. If the model predicts any patients with `num = 3` or `num = 4`, we know then that there is a high possibility that this patient has severe heart disease and should be checked with promptly. 

Future questions and research directions could include:

1. How can we improve the accuracy of the model further? Are there other machine learning algorithms or techniques that might yield better results with this dataset, such as logistic regression, support vector machines, or deep learning?
2. What additional features or predictors could be included in the analysis to improve the model's performance? Are there any other medical tests or patient characteristics that might be relevant in predicting heart disease?
3. How does the model's performance change when trained on a larger, more diverse dataset? Would collecting data from a wider variety of sources or locations help improve the model's generalizability?
4. How can we validate the model's performance in real-world settings? Can we set up clinical trials or partnerships with medical institutions to test the model's predictions against actual patient outcomes?
5. What are the ethical implications of using data science algorithms in healthcare, particularly in the context of predicting heart disease? How can we ensure that the models do not perpetuate biases or lead to unfair treatment of patients?
6. How can the model be integrated into existing healthcare workflows and systems? What changes might need to be made to the model or the way it is used to ensure that it can be effectively employed by medical professionals?

By addressing these questions in future studies, we can continue to refine the predictive model and better understand its potential applications and limitations in the context of heart disease diagnosis and treatment.

# References

* https://datasciencebook.ca/

* https://advstats.psychstat.org/book/mregression/selection.php

* https://rdrr.io/cran/GGally/man/ggpairs.html

* https://www.youtube.com/watch?v=0aTtMJO-pE4

* https://stackoverflow.com/a/50710989

* https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/grep

* https://ggplot2.tidyverse.org/reference/theme.html

* An Introduction to Statistical Learning (James et al. 2013)